In [1]:
import json
import re
import requests
import os
import zipfile
from io import BytesIO
from tqdm import tqdm

In [2]:
def download_and_extract_map(id, hash, download_url, ranked_playlist):
    # Download map from URL
    r = requests.get(download_url)
    zip = zipfile.ZipFile(BytesIO(r.content))

    # Find Info.dat and load it for song/author info and level .dat file names
    files = zip.namelist()
    info_file = None
    if 'Info.dat' in files:
        info_file = 'Info.dat'
    elif 'info.dat' in files:
        info_file = 'info.dat'
    else:
        print('can\'t find Info.dat')
        return
    mapset_info = json.load(zip.open(info_file))

    dir_name = f'{id} ({mapset_info["_songName"]} - {mapset_info["_levelAuthorName"]})'
    dir_name = re.sub(r'[\/:*?"<>|]', '', dir_name) # Conform to file naming rules
    save_path = os.path.join('./accsaber-maps/', dir_name)

    # Find file names of ranked difficulties and extract
    playlist_song = next((x for x in ranked_playlist['songs'] if x['hash'].lower() == hash))
    for difficulty in playlist_song['difficulties']:
        standard_characterisitic = next((x for x in mapset_info['_difficultyBeatmapSets'] if x['_beatmapCharacteristicName'] == 'Standard'))
        diff_name = difficulty['name'][0].upper() + difficulty['name'][1:]
        file = next((x['_beatmapFilename'] for x in standard_characterisitic['_difficultyBeatmaps'] if x['_difficulty'] == diff_name))
        zip.extract(file, path=save_path)
    
    # Store hash and save Info.dat
    mapset_info['_hash'] = hash
    with open(os.path.join(save_path, 'Info.dat'), 'w') as f:
        f.write(json.dumps(mapset_info))
    
    zip.close()

In [3]:
print('AccSaber Ranked Map Downloader!')
r = requests.get('https://api.accsaber.com/playlists/overall')
playlist = json.load(BytesIO(r.content))

all_hashes = list(map(lambda x: x['hash'].lower(), playlist['songs'])) # Reduce playlist to hash list
hashes_2d = [all_hashes[i:i + 50] for i in range(0, len(all_hashes), 50)] # Split into up to 50 long hash arrays

# Get info including download links for maps
beatsaver_maps = []
for hashes in hashes_2d:
    r = requests.get(f'https://api.beatsaver.com/maps/hash/{"%2C".join(hashes)}')
    maps = json.loads(r.text)
    beatsaver_maps += maps.values()

# Filter out reupped maps
returned_hashes = [x['versions'][0]['hash'].lower() for x in beatsaver_maps]
beatsaver_maps = [x for x in beatsaver_maps if x['versions'][0]['hash'].lower() in all_hashes]

# Download and extract non reupped maps
print('Downloading non reupped maps...')
for beatsaver_map in tqdm(beatsaver_maps):
    try:
        download_and_extract_map(beatsaver_map['id'], beatsaver_map['versions'][0]['hash'].lower(), beatsaver_map['versions'][0]['downloadURL'], playlist)
    except:
        print(f'Error downloading {beatsaver_map["id"]}')

# Download and extract reupped maps
print('Downloading reupped maps (may not work)...')
reupped_hashes = [x for x in all_hashes if x not in returned_hashes]
for hash in tqdm(reupped_hashes):
    try:
        r = requests.get(f'https://api.beatsaver.com/maps/hash/{hash}')
        id = json.loads(r.text)['id']
        download_url = f'http://cdn.beatsaver.com/{hash}.zip'
        download_and_extract_map(id, hash, download_url, playlist)
    except:
        print(f'Error downloading {hash}')

AccSaber Ranked Map Downloader!


100%|██████████| 118/118 [01:45<00:00,  1.12it/s]


100%|██████████| 3/3 [00:06<00:00,  2.02s/it]
